In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [27]:
import src.config as config
from src.inference import (
    get_feature_store
)

import pytz
from datetime import datetime, timezone, timedelta

def get_current_est_hour_ceiled():
    """Get current EST time ceiled to the next hour"""
    current = datetime.now(pytz.timezone("US/Eastern"))
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_est_hour_ceiled()
print(f"Current EST hour (ceiled): {current_hour}")

Current EST hour (ceiled): 2025-05-04 14:00:00-04:00


In [28]:
def fetch_predictions():
    current_hour = pd.Timestamp.now(tz='US/Eastern').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.start_hour == current_hour)

    return query.read()

In [29]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-05-04 13:45:29,764 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-04 13:45:29,768 INFO: Initializing external client
2025-05-04 13:45:29,769 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-04 13:45:30,549 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635


In [30]:
df= query.read()
df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.41s) 


,start_station_id,predicted_demand,start_hour
0,6822.089844,18.0,2025-05-04 16:00:00+00:00
1,5905.140137,21.0,2025-05-04 16:00:00+00:00
2,6140.049805,21.0,2025-05-04 16:00:00+00:00
3,6822.089844,9.0,2025-05-04 18:00:00+00:00
4,5905.140137,23.0,2025-05-04 18:00:00+00:00
5,6140.049805,14.0,2025-05-04 18:00:00+00:00


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype                  
---  ------            --------------  -----                  
 0   start_station_id  6 non-null      float32                
 1   predicted_demand  6 non-null      float64                
 2   start_hour        6 non-null      datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float32(1), float64(1)
memory usage: 252.0 bytes


In [32]:
df["start_hour"] = df["start_hour"].dt.tz_convert("US/Eastern")
df

,start_station_id,predicted_demand,start_hour
0,6822.089844,18.0,2025-05-04 12:00:00-04:00
1,5905.140137,21.0,2025-05-04 12:00:00-04:00
2,6140.049805,21.0,2025-05-04 12:00:00-04:00
3,6822.089844,9.0,2025-05-04 14:00:00-04:00
4,5905.140137,23.0,2025-05-04 14:00:00-04:00
5,6140.049805,14.0,2025-05-04 14:00:00-04:00


In [33]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current EST time and round up to next hour
    now = datetime.now(pytz.timezone("US/Eastern"))
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.start_hour >= current_date) &
    #     (fg.start_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['start_hour'] == hour_str]
    df_hour["start_hour"] = df["start_hour"].dt.tz_convert("US/Eastern")

    print(f"Current EST time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-05-04 13:45:32,848 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-04 13:45:32,854 INFO: Initializing external client
2025-05-04 13:45:32,855 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-04 13:45:33,682 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Current EST time: 2025-05-04 13:45:32.848250-04:00
Next hour: 2025-05-04 14:00:00-04:00
Querying for date range: 2025-05-04 to 2025-05-05
Filtering for hour: 2025-05-04 14:00
Found 3 records


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [34]:
now = datetime.now(pytz.timezone("US/Eastern"))

In [35]:
predictions

,start_station_id,predicted_demand,start_hour
3,6822.089844,9.0,2025-05-04 14:00:00-04:00
4,5905.140137,23.0,2025-05-04 14:00:00-04:00
5,6140.049805,14.0,2025-05-04 14:00:00-04:00
